Set up

In [17]:
import psycopg2
import os
import pandas as pd

In [3]:
# Set the working directory
current_dir = os.getcwd()
print('current_dir: ',current_dir)
parent_dir = os.path.dirname(current_dir)

os.chdir(parent_dir)

print('Current working directory:', os.getcwd())

current_dir:  c:\Users\Utente\Documents\GitHub\SE4GEO-Lab\db_helper
Current working directory: c:\Users\Utente\Documents\GitHub\SE4GEO-Lab


In [ ]:
from se4g_helper import download_request, build_dataframe, update_dataset, update_dashboard_dataset
# Download and get the dataframe file name
dir = download_request(folder_out = 'data_prova')

# Build the dataframe with the required structure
df = build_dataframe(dir, folder_out = 'data_prova')

# Update the final dataset
new_df = update_dataset(df, folder_out = 'data_prova')
# Update the dashboard dataset 
update_dashboard_dataset(df,folder_out = 'data_prova')

In [15]:
def insert_data(table_name, conn, df):
    cur = conn.cursor()
    
    # Specify the table name and column names
    df_columns = ['station_code', 
                'station_name', 
                'station_altitude', 
                'network_countrycode', 
                'pollutant', 
                'value_datetime_begin',
                'value_datetime_end',
                'value_datetime_updated',
                'value_numeric']

    # Iterate over the DataFrame rows and insert data row by row
    for _, row in df.iterrows():
        # Generate the SQL INSERT statement
        insert_statement = f"INSERT INTO {table_name} ({', '.join(df_columns)}) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
        values = tuple(row[col] for col in df_columns)

        # Execute the INSERT statement
        cur.execute(insert_statement, values)

    
    # Commit the changes
    conn.commit()
    
    # Close the cursor and the connection
    cur.close()
    conn.close()

ip = '192.168.30.19'

# Connect to the database
conn = psycopg2.connect(
    host = 'localhost',
    database = "se4g",
    user = "postgres",
    password = "carIs3198"
)

# Call the insert_data function with the table name and connection
insert_data('se4g_pollution', conn, new_df)


In [19]:
# Connect to the database
conn = psycopg2.connect(
    host = 'localhost',
    database = "se4g",
    user = "postgres",
    password = "carIs3198"
)

query = "SELECT * FROM se4g_pollution"
df = pd.read_sql_query(query, conn)

conn.close()

print(df)

C:\Users\Utente\AppData\Local\Temp\ipykernel_10752\839016635.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


       station_code        station_name  station_altitude network_countrycode  \
0           AD0942A  Escaldes-Engordany            1080.0                  AD   
1           AD0942A  Escaldes-Engordany            1080.0                  AD   
2           AD0942A  Escaldes-Engordany            1080.0                  AD   
3           AD0942A  Escaldes-Engordany            1080.0                  AD   
4           AD0942A  Escaldes-Engordany            1080.0                  AD   
...             ...                 ...               ...                 ...   
826323      SE0117A   Visby Österväg 17              44.0                  SE   
826324      SE0117A   Visby Österväg 17              44.0                  SE   
826325      SE0117A   Visby Österväg 17              44.0                  SE   
826326      SE0117A   Visby Österväg 17              44.0                  SE   
826327      SE0117A   Visby Österväg 17              44.0                  SE   

       pollutant       valu

In [ ]:
cur.execute('''UPDATE pollutant_detection
 (station_code TEXT , 
 name_pollutant TEXT, 
 data_time_begin DATE, 
 data_time_end DATE,
 data_time_update DATE,
 value_pollutant DECIMAL,
 PRIMARY KEY (name_pollutant, station_code, data_time_begin,data_time_end ))''')